In [19]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
import mlflow

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [7]:
client.create_experiment("my-experiment")

'2'

In [24]:
runs = client.search_runs(
  experiment_ids=["1"],
  filter_string="",
  run_view_type=ViewType.ACTIVE_ONLY,
  max_results=5,
  order_by=["metrics.rmse ASC"])

In [25]:
runs[-1].info.run_id

'bcf573a33d044be69956d57596b80fd9'

In [27]:
runs[-1].data.metrics["rmse"]

6.436693422952134

In [20]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [ ]:
model_uri = f"runs:/{runs[-1].info.run_id}/model"
run_id = runs[-1].info.run_id
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor", run_id=run_id)